In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install h5py scikit-learn matplotlib

import h5py
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
import matplotlib.pyplot as plt


In [3]:
import h5py
import numpy as np

h5_path = "/content/drive/MyDrive/Codigos_Fallos/Carpeta_Dataset/dataset_sync.h5"

X_vib, X_acoustic, X_temp, X_current, y = [], [], [], [], []

with h5py.File(h5_path, "r") as f:
    for cond in f.keys():
        g = f[cond]
        try:
            # Vibración
            v1 = g["vibration"]["accel_1"]["signal_sync"][:].astype(np.float32)
            v2 = g["vibration"]["accel_2"]["signal_sync"][:].astype(np.float32)
            if len(v1) == 0 or len(v2) == 0:
                continue
            min_len_vib = min(len(v1), len(v2))
            vib = np.stack([v1[:min_len_vib], v2[:min_len_vib]], axis=0)

            # Acústico
            if "acoustic" in g:
                a = g["acoustic"]["signal_sync"][:].astype(np.float32)
                if len(a) == 0:
                    a = np.zeros(min_len_vib, dtype=np.float32)
            else:
                a = np.zeros(min_len_vib, dtype=np.float32)

            # Temperatura
            if "temperature" in g["external"]:
                tmp = g["external"]["temperature"]["signal_sync"][:].astype(np.float32)
                if len(tmp) == 0:
                    tmp = np.zeros(min_len_vib, dtype=np.float32)
            else:
                tmp = np.zeros(min_len_vib, dtype=np.float32)

            # Corriente
            if "current" in g["external"]:
                cur = g["external"]["current"]["signal_sync"][:].astype(np.float32)
                if len(cur) == 0:
                    cur = np.zeros(min_len_vib, dtype=np.float32)
            else:
                cur = np.zeros(min_len_vib, dtype=np.float32)

            # Ajustar longitud mínima dentro de cada condición
            min_len_group = min(min_len_vib, len(a), len(tmp), len(cur))
            vib = vib[:, :min_len_group]
            a   = a[:min_len_group]
            tmp = tmp[:min_len_group]
            cur = cur[:min_len_group]

            # Guardar
            X_vib.append(vib)
            X_acoustic.append(a)
            X_temp.append(tmp)
            X_current.append(cur)
            y.append(cond)

        except Exception as e:
            print(f"Saltando {cond} por error: {e}")

# Recorte global con la mínima longitud
min_len_all = min(
    min(x.shape[1] for x in X_vib),
    min(len(a) for a in X_acoustic),
    min(len(t) for t in X_temp),
    min(len(c) for c in X_current)
)

X_vib      = np.array([x[:, :min_len_all] for x in X_vib])
X_acoustic = np.array([a[:min_len_all] for a in X_acoustic])
X_temp     = np.array([t[:min_len_all] for t in X_temp])
X_current  = np.array([c[:min_len_all] for c in X_current])
y          = np.array(y)

print("Shapes después de recorte global:")
print("Vib:", X_vib.shape)
print("Acoustic:", X_acoustic.shape)
print("Temp:", X_temp.shape)
print("Current:", X_current.shape)
print("y:", y.shape)


Shapes después de recorte global:
Vib: (45, 2, 1536000)
Acoustic: (45, 1536000)
Temp: (45, 1536000)
Current: (45, 1536000)
y: (45,)


In [4]:
with h5py.File(h5_path, "r") as f:
    cond = list(f.keys())[0]  # toma la primera condición
    print("Claves en external de", cond, ":")
    print(list(f[cond]["external"].keys()))


Claves en external de 0Nm_BPFI_03 :
['current', 'extra', 'temperature']


In [7]:
import h5py
import numpy as np
import pandas as pd

h5_path = "/content/drive/MyDrive/Codigos_Fallos/Carpeta_Dataset/dataset_sync.h5"

with h5py.File(h5_path, "r") as f:
    cond = list(f.keys())[0]  # primera condición
    print("Condición:", cond)

    # cargamos el bloque de corrientes
    currents = f[cond]["external"]["current"]["signal_sync"][:].astype(np.float32)
    print("Shape original:", currents.shape)

    # reorganizamos en 3 columnas (U, V, W)
    currents = currents.reshape(-1, 3)
    print("Shape reorganizado:", currents.shape)

    # separar fases
    Iu, Iv, Iw = currents[:, 0], currents[:, 1], currents[:, 2]

    # opcional: DataFrame para fácil manejo
    df_currents = pd.DataFrame({
        "U": Iu,
        "V": Iv,
        "W": Iw
    })

print(df_currents.head())


Condición: 0Nm_BPFI_03
Shape original: (1536492,)
Shape reorganizado: (512164, 3)
          U         V         W
0  1.894377  2.128889  2.373001
1  2.087747  2.393572  2.555399
2  2.471743  2.160432  2.171403
3  2.211174  2.490943  2.418258
4  2.497800  2.851625  2.755626


In [8]:
with h5py.File(h5_path, "r") as f:
    cond = list(f.keys())[0]

    temp = f[cond]["external"]["temperature"]["signal_sync"][:].astype(np.float32)
    print("Shape original temp:", temp.shape)

    # reorganizar en 2 columnas (A, B)
    temp = temp.reshape(-1, 2)
    Ta, Tb = temp[:, 0], temp[:, 1]

    print("Temp A primeras 10:", Ta[:10])
    print("Temp B primeras 10:", Tb[:10])


Shape original temp: (1536492,)
Temp A primeras 10: [0.94946295 0.8821538  1.1252909  0.7269307  0.7516565  0.5263769
 0.7942398  0.45769408 0.5469817  0.06070749]
Temp B primeras 10: [0.9261108  1.1939737  0.985178   0.93710005 0.5250032  0.6761054
 0.7502828  0.41785806 0.2791188  0.08131234]


In [18]:
# Normalización z-score
def normalize(X):
    mean = X.mean(axis=(0,2), keepdims=True)
    std  = X.std(axis=(0,2), keepdims=True) + 1e-8
    return (X - mean) / std

X_acoustic = X_acoustic[:, np.newaxis, :]
X_temp     = X_temp[:, np.newaxis, :]
X_current  = X_current[:, np.newaxis, :]
X_base     = np.concatenate([X_vib, X_acoustic], axis=1)

X_base    = normalize(X_base)
X_temp    = normalize(X_temp)
X_current = normalize(X_current)

# Etiquetas
unique_labels = sorted(set(y))
label_map = {name: idx for idx, name in enumerate(unique_labels)}
y_idx = np.array([label_map[name] for name in y])
num_classes = len(unique_labels)
print("Número de clases:", num_classes)

# Dataset
class MotorDatasetHybrid(Dataset):
    def __init__(self, X_base, X_temp, X_current, y, window_size=2048, step=2048):
        self.X_base, self.X_temp, self.X_current, self.y = X_base, X_temp, X_current, y
        self.window_size, self.step, self.indices = window_size, step, []
        for i in range(len(X_base)):
            n = X_base[i].shape[1]
            for start in range(0, n-window_size+1, step):
                self.indices.append((i,start,start+window_size))
    def __len__(self): return len(self.indices)
    def __getitem__(self, idx):
        i,s,e = self.indices[idx]
        return (
            torch.tensor(self.X_base[i][:,s:e],dtype=torch.float32),
            torch.tensor(self.X_temp[i][:,s:e],dtype=torch.float32),
            torch.tensor(self.X_current[i][:,s:e],dtype=torch.float32),
            torch.tensor(self.y[i],dtype=torch.long)
        )

dataset = MotorDatasetHybrid(X_base,X_temp,X_current,y_idx)
train_size = int(0.8*len(dataset))
val_size   = len(dataset)-train_size
train_dataset, val_dataset = random_split(dataset,[train_size,val_size])
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
val_loader   = DataLoader(val_dataset,batch_size=32,shuffle=False)


Número de clases: 45


In [19]:
class FuzzyLayerInput(nn.Module):
    """Fuzzy para temperatura (entrada externa)"""
    def __init__(self, n_sets=3, out_dim=64):
        super(FuzzyLayerInput, self).__init__()
        self.n_sets = n_sets
        self.centers = nn.Parameter(torch.linspace(-1, 1, n_sets))
        self.sigmas  = nn.Parameter(torch.ones(n_sets)*0.5)
        self.conv = nn.Sequential(
            nn.Conv1d(n_sets, 16, kernel_size=5, stride=2, padding=2),
            nn.ReLU(), nn.AdaptiveAvgPool1d(16)
        )
        self.fc = nn.Linear(16*16, out_dim)
    def membership(self, x, c, s):
        return torch.exp(-0.5 * ((x-c)/s)**2)
    def forward(self, x):
        B, _, L = x.shape
        sets = [self.membership(x.squeeze(1), self.centers[j], self.sigmas[j]).unsqueeze(1) for j in range(self.n_sets)]
        fuzzy_out = torch.cat(sets, dim=1)
        feat = self.conv(fuzzy_out).flatten(1)
        return self.fc(feat)

class FuzzyLayerModulator(nn.Module):
    """Fuzzy para corriente (modulación intermedia)"""
    def __init__(self, n_sets=3, feat_dim=32):
        super(FuzzyLayerModulator, self).__init__()
        self.n_sets=n_sets
        self.centers=nn.Parameter(torch.linspace(-1,1,n_sets))
        self.sigmas=nn.Parameter(torch.ones(n_sets)*0.5)
        self.conv=nn.Sequential(
            nn.Conv1d(n_sets,16,kernel_size=5,stride=2,padding=2),
            nn.ReLU(),nn.AdaptiveAvgPool1d(16)
        )
        self.fc_gamma=nn.Linear(16*16,feat_dim)
        self.fc_beta=nn.Linear(16*16,feat_dim)
    def membership(self,x,c,s):
        return torch.exp(-0.5*((x-c)/s)**2)
    def forward(self,x):
        B,_,L=x.shape
        sets=[self.membership(x.squeeze(1),self.centers[j],self.sigmas[j]).unsqueeze(1) for j in range(self.n_sets)]
        fuzzy_out=torch.cat(sets,dim=1)
        feat=self.conv(fuzzy_out).flatten(1)
        return self.fc_gamma(feat), self.fc_beta(feat)


In [20]:
class CNN_FuzzyHybrid(nn.Module):
    """
    Combina:
    - Temperatura (externa) como entrada fuzzy en paralelo.
    - Corriente (interna) como modulador intermedio.
    """
    def __init__(self, num_classes):
        super(CNN_FuzzyHybrid, self).__init__()

        # CNN base
        self.conv1 = nn.Sequential(
            nn.Conv1d(3, 16, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm1d(16), nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(16, 32, kernel_size=5, stride=2, padding=2),
            nn.BatchNorm1d(32), nn.ReLU()
        )
        self.pool = nn.AdaptiveAvgPool1d(64)

        # Fuzzy corriente (modulación)
        self.fuzzy_mod_cur = FuzzyLayerModulator(n_sets=3, feat_dim=32)

        # Fuzzy temperatura (entrada)
        self.fuzzy_temp = FuzzyLayerInput(n_sets=3, out_dim=64)

        # Clasificación final
        self.fc = nn.Sequential(
            nn.Linear(32*64 + 64, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x_base, x_temp, x_cur):
        out = self.conv1(x_base)
        out = self.conv2(out)

        # Modulación con corriente
        gamma, beta = self.fuzzy_mod_cur(x_cur)
        out = gamma.unsqueeze(-1)*out + beta.unsqueeze(-1)

        out_cnn = self.pool(out).flatten(1)
        feat_temp = self.fuzzy_temp(x_temp)

        feat_all = torch.cat([out_cnn, feat_temp], dim=1)
        return self.fc(feat_all)


In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN_FuzzyHybrid(num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
best_val_acc, patience, patience_counter = 0.0, 10, 0
best_model_path = "best_model_fuzzy_hybrid.pth"

for epoch in range(50):  # máximo de 50 épocas
    model.train()
    train_loss, train_correct, total = 0.0, 0, 0
    for xb, xt, xc, yb in train_loader:
        xb, xt, xc, yb = xb.to(device), xt.to(device), xc.to(device), yb.to(device)
        optimizer.zero_grad()
        outputs = model(xb, xt, xc)
        loss = criterion(outputs, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*yb.size(0)
        train_correct += (outputs.argmax(1)==yb).sum().item()
        total += yb.size(0)
    train_acc = train_correct/total

    # Validación
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    with torch.no_grad():
        for xb, xt, xc, yb in val_loader:
            xb, xt, xc, yb = xb.to(device), xt.to(device), xc.to(device), yb.to(device)
            outputs = model(xb, xt, xc)
            loss = criterion(outputs, yb)
            val_loss += loss.item()*yb.size(0)
            val_correct += (outputs.argmax(1)==yb).sum().item()
            val_total += yb.size(0)
    val_acc = val_correct/val_total

    print(f"Epoch {epoch+1:02d} | Train Loss {train_loss/total:.4f} Acc {train_acc:.4f} | Val Acc {val_acc:.4f}")

    # Early stopping
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        torch.save(model.state_dict(), best_model_path)
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping activado")
            break

print(f"Entrenamiento finalizado. Mejor val_acc={best_val_acc:.4f}, modelo en {best_model_path}")


Epoch 01 | Train Loss 1.2346 Acc 0.5512 | Val Acc 0.7807
Epoch 02 | Train Loss 0.7055 Acc 0.7018 | Val Acc 0.8212
Epoch 03 | Train Loss 0.6573 Acc 0.7196 | Val Acc 0.8031
Epoch 04 | Train Loss 0.6356 Acc 0.7265 | Val Acc 0.8680
Epoch 05 | Train Loss 0.5949 Acc 0.7442 | Val Acc 0.8401
Epoch 06 | Train Loss 0.5816 Acc 0.7509 | Val Acc 0.8468
Epoch 07 | Train Loss 0.5663 Acc 0.7576 | Val Acc 0.8231
Epoch 08 | Train Loss 0.5543 Acc 0.7606 | Val Acc 0.8471
Epoch 09 | Train Loss 0.5542 Acc 0.7602 | Val Acc 0.9107
Epoch 10 | Train Loss 0.5291 Acc 0.7730 | Val Acc 0.8561
Epoch 11 | Train Loss 0.5213 Acc 0.7728 | Val Acc 0.8713
Epoch 12 | Train Loss 0.5219 Acc 0.7743 | Val Acc 0.8639
Epoch 13 | Train Loss 0.5044 Acc 0.7836 | Val Acc 0.9071
Epoch 14 | Train Loss 0.4955 Acc 0.7851 | Val Acc 0.8674
Epoch 15 | Train Loss 0.5194 Acc 0.7770 | Val Acc 0.8754
Epoch 16 | Train Loss 0.5002 Acc 0.7870 | Val Acc 0.8785
Epoch 17 | Train Loss 0.4850 Acc 0.7917 | Val Acc 0.8926
Epoch 18 | Train Loss 0.4876 Ac